<a href="https://colab.research.google.com/github/mheidari98/Movie-Recommender-Systems/blob/main/LightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import drive
drive.mount('/content/gdrive')

!pip install lightfm

In [23]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank
from lightfm.data import Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix 
import time
from lightfm.evaluation import auc_score
import pickle
import re

In [2]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv( Dataset_path + "ml-latest/ratings.csv")

### just keep rating year between 1995, 2001

In [3]:
from datetime import datetime

# strftime('%Y-%m-%d %H:%M:%S')
ratings['year_rated'] = ratings['timestamp'].apply(lambda x: int( datetime.fromtimestamp( x ).strftime('%Y') ) )

ratings = ratings[ ratings['year_rated'].between(1995, 2001)].copy()

ratings.reset_index(drop=True, inplace=True)

In [10]:
print( f"number of Ratings : { ratings.shape[0] }")
print( f"number of movies : { ratings.groupby('movieId').count().shape[0] }")
print( f"number of users : { ratings.groupby('userId').count().shape[0] }")
print( f"range of rating : ( { np.min( ratings.groupby('rating').count().index )}, {np.max( ratings.groupby('rating').count().index )})  ")

number of Ratings : 7329482
number of movies : 4937
number of users : 103827
range of rating : ( 1.0, 5.0)  


In [32]:
counts1 = ratings['userId'].value_counts()
ratings = ratings[ratings['userId'].isin(counts1[counts1 >= 20].index)]
counts = ratings['rating'].value_counts()
ratings = ratings[ratings['rating'].isin(counts[counts >= 20].index)]
ratings.shape

(6958466, 3)

### LightFM 

In [18]:
del ratings['year_rated']
del ratings['timestamp']
#del CleanedRatings['timestamp']


In [33]:
#Generating ratings matrix from ratings table
ratings_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
userID = ratings_matrix.index
movieId = ratings_matrix.columns
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)
print(ratings_matrix.shape)
ratings_matrix.head()

(65785, 4913)


movieId,1,2,3,4,5,6,7,8,9,10,...,5717,6184,6425,6460,6474,6531,6677,6910,6918,8739
userId,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
8,0,0,3,0,3,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,3,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [34]:
def _shuffle(uids, iids, data, random_state):

    shuffle_indices = np.arange(len(uids))
    random_state.shuffle(shuffle_indices)

    return (uids[shuffle_indices],
            iids[shuffle_indices],
            data[shuffle_indices])

def random_train_test_split(interactions_df, test_percentage=0.25,random_state=None):

    interactions = csr_matrix(interactions_df.values)
    if random_state is None:
        random_state = np.random.RandomState()

    interactions = interactions.tocoo()

    shape = interactions.shape
    uids, iids, data = (interactions.row,
                        interactions.col,
                        interactions.data)

    uids, iids, data = _shuffle(uids, iids, data, random_state)

    cutoff = int((1.0 - test_percentage) * len(uids))

    train_idx = slice(None, cutoff)
    test_idx = slice(cutoff, None)

    train = coo_matrix((data[train_idx],
                           (uids[train_idx],
                            iids[train_idx])),
                          shape=shape,
                          dtype=interactions.dtype)
    test = coo_matrix((data[test_idx],
                          (uids[test_idx],
                           iids[test_idx])),
                         shape=shape,
                         dtype=interactions.dtype)

    return train, test


In [35]:
train,test = random_train_test_split(ratings_matrix)

### Warp

In [30]:
model01 = LightFM(loss='warp',
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

In [31]:
start_time = time.time()

model01 = model01.fit(train,
                  epochs=10,
                  num_threads=16, verbose=True)

print("--- Run time:  {} mins ---".format((time.time() - start_time)/60))

Epoch:   0%|          | 0/10 [09:26<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
train_precision = precision_at_k(model01, train, k=10).mean()
test_precision = precision_at_k(model01, test, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))


In [36]:
train_aucscore = auc_score(model01, train).mean()
test_aucscore = auc_score(model01, test).mean()
print('auc_score: train %.2f, test %.2f.' % (train_aucscore, test_aucscore))

auc_score: train 0.53, test 0.50.


In [52]:
pred01 = model01.predict_rank(test)

In [54]:
print('MSE:',mean_squared_error(test.toarray(), pred01.toarray()))
print('MAE:',mean_absolute_error(test.toarray(), pred01.toarray()))
print('RMSE:',np.sqrt(mean_squared_error(test.toarray(), pred01.toarray())))

MSE: 9562.134629953864
MAE: 3.2270124445349984
RMSE: 97.78616788663857


### Bpr

In [39]:
model02 = LightFM(loss='bpr',
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

In [40]:
model02 = model02.fit(train,
                  epochs=10,
                  num_threads=16, verbose=True)

Epoch: 100%|██████████| 10/10 [16:13<00:00, 97.39s/it] 


In [60]:
train_precision = precision_at_k(model02, train, k=10).mean()
test_precision = precision_at_k(model02, test, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.02, test 0.00.


In [58]:
train_aucscore = auc_score(model02, train).mean()
test_aucscore = auc_score(model02, test).mean()
print('auc_score: train %.2f, test %.2f.' % (train_aucscore, test_aucscore))

auc_score: train 0.94, test 0.88.


In [55]:
pred02 = model02.predict_rank(test)

In [59]:
print('MSE:',mean_squared_error(test.toarray(), pred02.toarray()))
print('MAE:',mean_absolute_error(test.toarray(), pred02.toarray()))
print('RMSE:',np.sqrt(mean_squared_error(test.toarray(), pred02.toarray())))

MSE: 1680.8388972835778
MAE: 1.1013205051841477
RMSE: 40.998035285652136
